### This Notebook shows how a **Backer** can writte KERI Key Events to Cardano blockchain and how a **Watcher** can retrieve the full Key Event Log (KEL) from the blockchain.
This code use Blockfrost API to intercat with Cardano Nodes

In [ ]:
from blockfrost import BlockFrostApi, ApiError, ApiUrls
from pycardano import * 
import os

First create a ProjectId in https://blockfrost.io for the network of your choice. In this case we are using the _Preview_ network.
Put your projectId in an evironmental variable or paste it in the code bellow.

In [ ]:
blockfrostProjectId = os.getenv('BLOCKFROST_PROJECT_ID')
blockfrostProjectId="xxx"
api = BlockFrostApi(
    project_id=blockfrostProjectId,
    base_url=ApiUrls.preview.value
)
network = Network.TESTNET
context = BlockFrostChainContext(blockfrostProjectId,network, ApiUrls.preview.value)

Generate payment and stake key pairs. You can Save and Load on this disk.

In [ ]:
payment_key_pair = PaymentKeyPair.generate()
payment_signing_key = payment_key_pair.signing_key
payment_verification_key = payment_key_pair.verification_key

stake_key_pair = StakeKeyPair.generate()
stake_signing_key = stake_key_pair.signing_key
stake_verification_key = stake_key_pair.verification_key

In [ ]:
# Save keys
payment_signing_key.save("payment.skey")
payment_verification_key.save("payment.vkey")
stake_signing_key.save("stake.skey")
stake_verification_key.save("stake.vkey")

In [ ]:
# Load keys
payment_signing_key = PaymentSigningKey.load("payment.skey")
payment_verification_key = PaymentVerificationKey.load("payment.vkey")
stake_signing_key = StakeSigningKey.load("stake.skey")
stake_verification_key = StakeVerificationKey.load("stake.vkey")

Generate the Stake Address and the Spending Address that the Backer will use to submit transactions with metadata to Cardano.
You can fund the Spending Address with test ADA at https://docs.cardano.org/cardano-testnet/tools/faucet"

In [ ]:
spending_addr = Address(payment_verification_key.hash(), stake_verification_key.hash(), network=network)
stake_addr = Address(payment_part=None, staking_part=stake_verification_key.hash(), network=network)
print("Stake address:", stake_addr.encode())
print("Spending Address:", spending_addr.encode())

Using Blockfrost API you can query ADA balances and UTXOs

In [ ]:
# Check address balance
address = api.address(
        address=spending_addr.encode())
for amount in address.amount:
    print(amount.quantity, amount.unit)

In [ ]:
# Check wallet balance
addresses = api.account_addresses(stake_addr.encode())
for addr in addresses:
    addrR = api.address(addr.address)
    for amount in addrR.amount:
        print(amount.quantity, amount.unit,":", addrR.address)


In [ ]:
# Get UTXOs
utxos = api.address_utxos(spending_addr.encode())
print(utxos)

Build and sign a transaction. The Key Event is added as transaction metadata.

In [ ]:
builder = TransactionBuilder(context)
builder.add_input_address(spending_addr)
builder.add_output(TransactionOutput(spending_addr,Value.from_primitive([1000000])))
builder.auxiliary_data = AuxiliaryData(Metadata(
            { 
                1: {
                    "v": "KERI10JSON000159_",
                    "t": "icp",
                    "d": "EDRc4HIqxKvCeafMxSMJM2tuQfkERkgtzUAAVLdk-bhj",
                    "i": "EDRc4HIqxKvCeafMxSMJM2tuQfkERkgtzUAAVLdk-bhj",
                    "s": "0",
                    "kt": "1",
                    "k": [
                    "DNnrwNUFl070TGU8wlYSZZ0WZ-V5WbDqXnBv-HkXjQ4x"
                    ],
                    "nt": "1",
                    "n": [
                    "EJ2BHzcravgL1BfgEbNo8V181WNvKprybAkae4QSLaG1"
                    ],
                    "bt": "1",
                    "b": [
                    "BBilc4-L3tFUnfM_wJr4S4OJanAv_VmF_dJNN6vkf2Ha"
                    ],
                    "c": [],
                    "a": []
                }
            }
        )
    )
signed_tx = builder.build_and_sign([payment_signing_key], change_address=spending_addr)
print(signed_tx.id)

Submit transaction to Cardano

In [ ]:
context.submit_tx(signed_tx.to_cbor())

Check transaction. It may takes time to get published

In [ ]:
tx = api.transaction(signed_tx.id)
print(tx)

In [ ]:
metadata = api.transaction_metadata(signed_tx.id)
print(metadata)

The Backer should submit transactions any time it receives a Key Event.
The Watcher can retrieve the full KEL from Cardano blockchain as follows:

In [ ]:
txs = api.address_transactions(spending_addr.encode())
for tx in txs:
    print(api.transaction_metadata(tx.tx_hash))